# Library Importation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
import joblib
from sklearn.metrics import roc_auc_score
import os
import random
import xgboost
import numpy as np
import matplotlib.pylab as plt

import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler

import category_encoders as ce


import category_encoders as ce
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
import logging
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer


# Data Importation

In [ ]:

InputSPI=pd.DataFrame()
for item in range(4):
    InputSPI=pd.concat([InputSPI,pd.read_csv('data-challenge-phme/data/SPI_training_'+str(item)+'.csv.zip')])
    
#InputSPI=cleaning_SPI(InputSPI)

InputAOI=pd.read_csv('data-challenge-phme/data/AOI_training.csv.zip')




# Data Training Preparation

In [3]:
spi=InputSPI.copy()
aoi=InputAOI.copy()

In [4]:
spi["ComponentID2"]=spi["ComponentID"]
spi["FigureID2"]=spi["FigureID"]

aoi['PinNumber']=aoi['PinNumber'].astype('Int64').astype(str)


In [6]:
spi_training=spi.merge(aoi,on=['PanelID','FigureID','ComponentID','PinNumber'],how="left")
spi_training=spi_training.drop_duplicates(subset=['PanelID','FigureID','ComponentID','PinNumber'])
Num_features_to_cat=['Volume(%)','Area(%)', 'OffsetX(%)','OffsetY(%)']
Num_features=['Shape(um)',
       'PosX(mm)', 'PosY(mm)','SizeX', 'SizeY']

Cat_features=['FigureID','PinNumber','PadID', 'PadType','Result']

spi_training['Target']=spi_training['MachineID'].isna()
spi_training=spi_training.dropna(subset=Num_features)

for feat in Num_features:
    spi_training[feat]=spi_training[feat].astype('float')
    
spi_training["ComponentID2"]=spi_training["ComponentID"].astype('category').cat.codes



# Training set Creation + Encoding with Catboost

In [ ]:
Train_df=spi_trainin.reset_index(drop=True)
#Train_df=Train_df.loc[Train_df.ComponentID!="BC1"]
#Test_df=Test_df.loc[Test_df.ComponentID!="BC1"]

X_train=Train_df
y_train=Train_df['Target']

encoder=ce.CatBoostEncoder(a=0.3)

AE_val=encoder.fit_transform(Train_df["ComponentID2"].copy(),y_train)
AE_val.columns=[x+"_encoded" for x in AE_val.columns.tolist()]
Train_df=pd.concat([Train_df,AE_val],axis=1)
features_x=[x for x in Train_df.columns if "_encoded" in x]
joblib.dump(encoder,"Cat_boost_step1")


X_train_final=Train_df[features_x].to_numpy()


# Optuna Hyperparametrisation

In [ ]:
ratio=np.sum(y_train==0)/np.sum(y_train==1)



def objective(trial: Trial,X,y) -> float:
    
    joblib.dump(study, 'study.pkl')

    param = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']), 
        'tree_method': trial.suggest_categorical('tree_method',['exact']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        #'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        'n_estimators': trial.suggest_categorical('n_estimators', [30,40,50,70,100,150,200]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'nthread' : -1,
        #0'feval': trial.suggest_categorical('feval',[f1_eval]),
        #'scale_pos_weight' : trial.suggest_categorical('scale_pos_weight', [ratio])
        
    }
    
    
    model = xgboost.XGBClassifier(**param,feval=f1_eval)
    
    return cross_val_score(model, X_train_final, y_train, cv=5,scoring=make_scorer(f1_score, pos_label=0)).mean()


In [ ]:
from sklearn.metrics import f1_score
import numpy as np

def f1_eval(y_pred, y_true):
    #y_true = dtrain.get_label()
    err = f1_score(y_true,y_pred,pos_label=0)
    return 'f1_err', err

In [ ]:
study = optuna.create_study(direction='maximize')
#study = optuna.create_study(direction='maximize', storage="sqlite:///xgb_optuna_home_credit.db", study_name="optuna_xgb_output_8")
study.optimize(lambda trial : objective(trial,X_train_final,y_train),n_trials= 100)

# Best trial model training

In [ ]:
Best_trial = study.best_trial.params
model = xgboost.XGBClassifier(**Best_trial)
model.fit(X_train_final,y_train)
confusion_matrix(y_train, model.predict(X_train_final),labels=[0,1])
cross_val_score(model, X_train_final, y_train, cv=5,scoring=make_scorer(f1_score, pos_label=0)).mean()

In [ ]:
model.save_model('xgboost_step1')

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

list_panel_id=InputAOI.PanelID.unique().tolist()

def cross_validation_panel_id_creation(list_panel_id,n_fold=5):
    Fold_list=[]
    random.shuffle(list_panel_id)
    #list_panel_id=joblib.load("list_panel_id")
    len_fold=(len(list_panel_id))//n_fold
    for fold in range(n_fold):
        Fold_list.append(list_panel_id[fold*len_fold:(fold+1)*len_fold])
    return(Fold_list)
Fold_list=cross_validation_panel_id_creation(list_panel_id)
#Final_df=Final_df.loc[Final_df['ComponentID2']!="BC1"]
f1_score_list=[]
list_confusion_matrix=[]
for Test_fold_indice in range(1):
    print(Test_fold_indice)
    Train_indice=[]
    count_indice_fit_pin_training=0
    for indices in range(len(Fold_list)):
        if indices!=Test_fold_indice:
            Train_indice=Train_indice+Fold_list[indices]
        else:
            Test_indices=Fold_list[indices]
            #Train_indice=Train_indice+Fold_list[indices]

    Train_df=spi_training.loc[spi_training["PanelID"].isin(Train_indice)].reset_index(drop=True)
    #Train_df=Train_df.loc[Train_df.ComponentID!="BC1"]
    Test_df=spi_training.loc[spi_training["PanelID"].isin(Test_indices)].reset_index(drop=True)
    #Test_df=Test_df.loc[Test_df.ComponentID!="BC1"]

    X_train=Train_df
    X_val=Test_df
    y_train=Train_df['Target']
    y_val=Test_df['Target']
    y_train=Train_df['Target']
    y_train=Train_df['Target']
    y_val=Test_df['Target']
    encoder=ce.CatBoostEncoder(a=0.3)

    AE_val=encoder.fit_transform(Train_df["ComponentID2"].copy(),y_train)
    AE_val.columns=[x+"_encoded" for x in AE_val.columns.tolist()]
    Train_df=pd.concat([Train_df,AE_val],axis=1)
    features_x=[x for x in Train_df.columns if "_encoded" in x]
    joblib.dump(encoder,"Cat_boost_step1")

    AE_val=encoder.transform(Test_df["ComponentID2"].copy())
    AE_val.columns=[x+"_encoded" for x in AE_val.columns.tolist()]
    Test_df=pd.concat([Test_df,AE_val],axis=1)

    features_x=["ComponentID2_encoded"]+Num_features_to_cat+Num_features
    
    X_train_final=Train_df[features_x].to_numpy()
    X_val_final=Test_df[features_x].to_numpy()
    
    xgb_cl =model
    xgb_cl.fit(X_train_final, y_train)


    res_train=xgb_cl.predict(X_train_final)
    res_test=xgb_cl.predict(X_val_final)

    X_train['Pred']=res_train
    X_val['Pred']=res_test
   
    
    y_train=X_train.groupby(["PanelID","FigureID","ComponentID"])['Target'].min()
    y_val=X_val.groupby(["PanelID","FigureID","ComponentID"])['Target'].min()


    res_train=X_train.groupby(["PanelID","FigureID","ComponentID"])['Pred'].min()
    res_test=X_val.groupby(["PanelID","FigureID","ComponentID"])['Pred'].min()




    print(confusion_matrix(y_train, res_train,labels=[0,1]))
    print(confusion_matrix(y_val,res_test,labels=[0,1]))

    print(f1_score(y_train, res_train,pos_label=0))
    print(f1_score(y_val, res_test,pos_label=0))
    f1_score_list.append(f1_score(y_val, res_test,pos_label=0))

0
